# Clase 3.5 - Persistencia de Datos

**Unidad:** 3 - Desarrollo Web con FastAPI  

## Objetivos de Aprendizaje

- [ ] Conectar con SQLite
- [ ] Crear modelos SQLAlchemy
- [ ] Implementar CRUD basico
- [ ] Usar dependencias para sesion BD

In [ ]:
!pip install fastapi sqlalchemy -q

from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

---

## 1. Configurar Base de Datos

In [ ]:
# SQLite en memoria para pruebas
DATABASE_URL = "sqlite:///./test.db"

engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

print("Base de datos configurada")

---

## 2. Definir Modelo

In [ ]:
class TareaModel(Base):
    __tablename__ = "tareas"

    id = Column(Integer, primary_key=True, index=True)
    titulo = Column(String(100), nullable=False)
    descripcion = Column(String(500))
    completada = Column(Boolean, default=False)

# Crear tablas
Base.metadata.create_all(bind=engine)
print("Tablas creadas")

---

## 3. Dependencia para Sesion

In [ ]:
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

---

## 4. CRUD con SQLAlchemy

In [ ]:
from fastapi import FastAPI, Depends
from pydantic import BaseModel
from sqlalchemy.orm import Session

app = FastAPI()

class TareaCreate(BaseModel):
    titulo: str
    descripcion: str = None

class TareaResponse(BaseModel):
    id: int
    titulo: str
    completada: bool

    class Config:
        from_attributes = True

@app.post("/tareas", response_model=TareaResponse)
def crear_tarea(tarea: TareaCreate, db: Session = Depends(get_db)):
    db_tarea = TareaModel(**tarea.dict())
    db.add(db_tarea)
    db.commit()
    db.refresh(db_tarea)
    return db_tarea

@app.get("/tareas", response_model=list[TareaResponse])
def listar_tareas(db: Session = Depends(get_db)):
    return db.query(TareaModel).all()

print("Endpoints CRUD creados")

---

## Resumen

| Concepto | Descripcion |
|----------|-------------|
| create_engine | Crear conexion BD |
| declarative_base | Clase base para modelos |
| SessionLocal | Fabrica de sesiones |
| Depends(get_db) | Inyectar sesion |